## Parse input

In [3]:
from collections import defaultdict, namedtuple

Coord = namedtuple('Coord', ['x', 'y'])
pts = []
with open('input.txt') as infile:
    for l in infile:
        x, y = l.strip().split(', ')
        pts.append(Coord(int(x), int(y)))

min_x = min([c.x for c in pts])
min_y = min([c.y for c in pts])
max_x = max([c.x for c in pts])
max_y = max([c.y for c in pts])

coords = [
    Coord(c.x - min_x, c.y - min_y)
    for c in pts
]
w = max_x - min_x + 1
h = max_y - min_y + 1

## Part 1 - breadth-first-search

In [65]:
grid = [[None] * h for _ in range(w)]

size = defaultdict(int)
disqualified = {-1}
fringe = {   
    coord: {owner}
    for owner, coord in enumerate(coords)
}
while fringe:
    newly_claimed = []
    for (x, y), claims in fringe.items():
        assert(grid[x][y] is None)
        owner = claims.pop() if len(claims) == 1 else -1
        grid[x][y] = owner
        size[owner] += 1
        newly_claimed.append((x, y, owner))

    fringe = defaultdict(set)
    for x, y, owner in newly_claimed:
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_x, new_y = x + dx, y + dy
            if not ((0 <= new_x < w) and (0 <= new_y < h)):
                disqualified.add(owner)
                continue
            if grid[new_x][new_y] is not None:
                continue
            fringe[(new_x, new_y)].add(owner)

max([(s, owner) for owner, s in size.items() if owner not in infinite])


(4398, 42)

In [ ]:
# Print out grid to help debug
for column in grid:
    vals = []
    for n in column:
        vals.append("%02d" % n)
    print(','.join(vals))

## Part 1 - brute force

In [67]:
size = defaultdict(int)
disqualified = {-1}
for x in range(w):
    for y in range(h):
        owner = None
        min_dist = h + w + 1
        for i, c in enumerate(coords):
            dist = abs(x - c.x) + abs(y - c.y)
            if dist < min_dist:
                min_dist = dist
                owner = i
            elif dist == min_dist:
                owner = -1
        size[owner] += 1
        if x in {0, w - 1} or y in {0, h - 1}:
            disqualified.add(owner)

In [71]:
max([(s,owner) for owner, s in size.items() if owner not in disqualified])

(4398, 42)

## Part 2

In [12]:
MAXD = 10000

total_nodes = len(coords)
nx = defaultdict(int)  # number of nodes in column x
ny = defaultdict(int)  # number of nodes in row y
for x, y in coords:
    nx[x] += 1
    ny[y] += 1

area = 0
left_nodes = 0
dy0 = sum(y for _, y in coords)  # total vertical-distance when y == 0
for x in range(w):
    if x == 0:
        dx = sum(x for x, _ in coords)
    else:
        # moving right 1 space increases distance by 1 for each node to the left
        # and decreases distance by 1 for each node to your right 
        dx += left_nodes - (total_nodes - left_nodes)
    left_nodes += nx.get(x, 0)
    dy_lim = MAXD - dx
    if dy_lim <= 0:
        continue
    
    dy = dy0
    above_nodes = 0
    for y in range(h):
        if dy < dy_lim:
            area += 1
        # moving down 1 space increases distance by 1 for each node above
        # and decreases distance by 1 for each node below
        above_nodes += ny.get(y, 0)
        dy += above_nodes - (total_nodes - above_nodes)

area

39560

## Maybe slightly more readable solution to Part 2

In [10]:
MAXD = 10000

total_nodes = len(coords)
nx = defaultdict(int)  # number of nodes in column x
ny = defaultdict(int)  # number of nodes in row y
for x, y in coords:
    nx[x] += 1
    ny[y] += 1

dx = [None] * w  # total horizontal-distance for a given x
dy = [None] * h  # total vertical-distance for a given y
dx[0] = sum([x for x, _ in coords])
dy[0] = sum([y for _, y in coords])

nodes_passed = nx.get(0,0)
for x in range(1, w):
    dx[x] = dx[x - 1] + nodes_passed - (total_nodes - nodes_passed)
    nodes_passed += nx.get(x, 0)
nodes_passed = ny.get(0,0)
for y in range(1, h):
    dy[y] = dy[y - 1] + nodes_passed - (total_nodes - nodes_passed)
    nodes_passed += ny.get(y, 0)

area = 0
for x in range(w):
    dy_lim = MAXD - dx[x]
    if dy_lim <= 0:
        continue
    area += sum(1 for dist in dy if dist < dy_lim)

area

39560